<a href="https://colab.research.google.com/github/jai-llm/TEXT2SQL/blob/main/1_Text2SQL_FineTune_Llama2GPTQv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Text2SQL LLaMA2GPTQ Fine-Tune
Runs in Free Google Colab needs T4 GPU to run.

In [1]:
!pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes
! pip install -q -U rouge_score

### Imports

In [2]:
import pandas as pd
import json
import torch
import os
import time

# In case Login Required For Model
# from huggingface_hub import login
# from dotenv import load_dotenv

from datasets import load_dataset, Dataset, load_metric, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print

In [3]:
# If Login Required for Model Access
# load_dotenv("/notebooks/.env")
# os.environ["TOKENIZERS_PARALLELISM"]="false"
# login(token=os.getenv("HUGGINGFACE_TOKEN"))

### Global Constants

In [4]:
model_id = "TheBloke/Llama-2-7B-GPTQ"
# model_id = "TheBloke/Llama-2-7b-Chat-GPTQ"
checkpoint_name ="SQL_llama2_gptq_7b_peftv1_"+time.strftime("%Y%m%d_%H%M%S")
OUT_DIR = "sql_gptq_training"

In [5]:
print(checkpoint_name)

SQL_llama2_gptq_7b_peftv1_20230930_043049

In [6]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"

### Common Functions

In [7]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [8]:
rouge = load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

<ipython-input-8-485d1ab45255>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [9]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

### Load and Check Data

In [10]:
# Load Training Data from Disk
dataset = load_from_disk(DATA_PATH + DS_DIR)

In [11]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [12]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['question', 'context', 'response', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['question', 'context', 'response', 'text', '__index_level_0__'],
    num_rows: 454
})

In [13]:
display(test_df.head(2))
display(test_df.shape)

,question,context,response,__index_level_0__,text
0,Show the name of track and the number of races...,"CREATE TABLE track (name VARCHAR, track_id VAR...","SELECT T2.name, COUNT(*) FROM race AS T1 JOIN ...",429,### Instruction:\n You are a powerful text-...
1,Show names of shops and the carriers of device...,"CREATE TABLE shop (Shop_Name VARCHAR, Shop_ID ...","SELECT T3.Shop_Name, T2.Carrier FROM stock AS ...",2907,### Instruction:\n You are a powerful text-...


(454, 5)

### FineTune GPTQ Model

In [14]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# LLM GPTQ Model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=quantization_config_loading,
                                             device_map="auto")

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [15]:
# Get Model Memory Footprint = ~4GB
print(model.get_memory_footprint()/1e9) # GB

3.9636992

In [16]:
model.config.use_cache = False
model.config.pretraining_tp = 1
# %%
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# needed for llama 2 tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
model.config.use_cache = False # silence the warnings. Please re-enable for inference!

trainable params: 8,388,608 || all params: 270,798,848 || trainable%: 3.097726619575575


In [17]:
args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100, # Change this if T4 Timesout
        learning_rate=2e-4,
        fp16=True, #use mixed precision training
        logging_steps=1,
        output_dir=OUT_DIR,
        overwrite_output_dir=True,
        optim="adamw_hf",
        save_strategy="epoch",
        report_to="none")

In [18]:
# set training arguments - Feel free to adapt it
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    warmup_steps=2,
    max_steps=5, #100, # Change this if T4 Timesout
    optim="adamw_hf",
    learning_rate=2e-4,
    fp16=True, #use mixed precision training
    # predict_with_generate=True, # Needed for Seq2Seq models only
    logging_steps=1, #500,
    save_strategy="epoch",
    #save_steps=1000,
    #eval_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=False, #True
    report_to="none"
)

In [19]:
trainer = SFTTrainer(
    model=model,
    args=args, # Training Only
    # args=training_args, # Evaluation
    # compute_metrics=compute_metrics, # uncomment for ROGUE Metrics
    train_dataset=dataset['train'],
    # eval_dataset = dataset['test'], # Evaluation Only
    peft_config=config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512)

In [20]:
# Takes ~20 mins to finetune
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.090800
2,2.143800
3,2.064400
4,1.921500
5,1.650600
6,1.519800
7,1.377300
8,1.148700
9,1.060400
10,0.882400


In [21]:
# To merge and save the model
output_dir = os.path.join(args.output_dir, checkpoint_name)
trainer.model.save_pretrained(output_dir)

In [22]:
# To perform inference on the test dataset example load the model from the checkpoint
persisted_model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda",
)

In [23]:
ID = 10
print('Question:')
display(dataset['test'][ID]['question'])
print('Context:')
display(dataset['test'][ID]['context'])
print('Response:')
display(dataset['test'][ID]['response'])

Question:

'Show ids for all students who have advisor 1121.'

Context:

'CREATE TABLE Student (StuID VARCHAR, Advisor VARCHAR)'

Response:

'SELECT StuID FROM Student WHERE Advisor = 1121'

In [24]:
text = test_df['text'][ID]
print(text)

### Instruction:
    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a
question and context regarding one or more tables.

    You must output the SQL query that answers the question.

    ### Input:
    Show ids for all students who have advisor 1121.
    ### Context:
    CREATE TABLE Student (StuID VARCHAR, Advisor VARCHAR)
    ### Response:

In [25]:
text = test_df['text'][ID]
inputs = tokenizer(text, return_tensors="pt").to('cuda')
generation_config = GenerationConfig(
    penalty_alpha=0.5,
    # do_sample = True,
    top_k=1,
    # temperature=0.1,
    repetition_penalty=1.2,
    max_new_tokens=180
)
start_time = perf_counter()
outputs = persisted_model.generate(**inputs, generation_config=generation_config)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))
output = parse(tokenizer.decode(outputs[0]))
result = {'response': output}
print(json.dumps(result))
end_time = perf_counter()
output_time = end_time - start_time
print(f"Time taken for inference: {round(output_time,2)} seconds")

{"response": "SELECT StuID FROM Student WHERE Advisor = \"1121\""}

Time taken for inference: 3.82 seconds

In [26]:
# display(test_df.head(2))
print('Data:  ' + test_df.loc[ID, 'response'])
print('LLM_:  ' + output)

Data:  SELECT StuID FROM Student WHERE Advisor = 1121

LLM_:  SELECT StuID FROM Student WHERE Advisor = "1121"